# Exploring Twitter Data with Vector Databases and RAG Systems

This tutorial introduces the creation of a vector database and the use of a retrieval-augmented generation (RAG) system to explore Twitter data interactively. 

Please check [LBSocial](www.lbsocial.net)  on how to collect Twitter data. 

## Set up a Database and API Keys

Create a [MongoDB](www.mongodb.com) cluster and store the connection string in a safe place, such as AWS Secrets Manager. 
- key name: `connection_string`
- key value: <`the connection string`>, you need to type the password
- secret name: `mongodb`


You also need to purchase and your [oepnai](https://openai.com/) api key in AWS Secrets Manager:
- key name: `api_key`
- key value: <`your openai api key`>
- secret name: `openai`

## Install Python Libraries

- pymongo: manage the MongoDB database
- openai: create embeddings and resonpses 

In [1]:
pip install pymongo openai -q

Note: you may need to restart the kernel to use updated packages.


## Secrets Manager Function

In [2]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Import Python Libraries and Credentials

In [3]:
import pymongo
from pymongo import MongoClient
import json
import re
import os
from openai import OpenAI
openai_api_key  = get_secret('openai_1')['api_key']
client = OpenAI(api_key=openai_api_key)


mongodb_connect = get_secret('mongodb')['connection_string']


## Connect to the MongoDB cluster

In [4]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection

## Utility Funcitons

- the `clean_tweet` function removes URLs in tweets
- the `get_embedding` function use openai to create tweet embeddings
- the `vector_search` function return relevent tweets based on a query

In [5]:
def clean_tweet(text):
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(url_pattern, '', text)

In [6]:
embedding_model= 'text-embedding-3-small'

def get_embedding(text):

    try:
        embedding = client.embeddings.create(input=text, model=embedding_model).data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

In [7]:
def vector_search(query):

    query_embedding = get_embedding(query)
    if query_embedding is None:
        return "Invalid query or embedding generation failed."
    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "tweet_vector",
                "queryVector": query_embedding,
                "path": "tweet.embedding",
                "numCandidates": 1000,  # Number of candidate matches to consider
                "limit": 10  # Return top 10 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "tweet.text": 1 # return tweet text

            }
        }
    ]

    results = tweet_collection.aggregate(pipeline)
    return list(results)

## Tweets Embedding 

For more about text embeddings please read [Introducing text and code embeddings](https://openai.com/index/introducing-text-and-code-embeddings/)

In [8]:
from tqdm.auto import tqdm
tweets = tweet_collection.find()

for tweet in tqdm(list(tweets)):
    try:
        tweet_embedding = get_embedding(clean_tweet(tweet['tweet']['text']))
    #     print(tweet_embedding)

        tweet_collection.update_one(
            {'tweet.id':tweet['tweet']['id']},
            {"$set":{'tweet.embedding':tweet_embedding}}
        )
    except:
        print(f"""error in embedding tweet {tweet['tweet']['id']}""")
        pass


  0%|          | 0/97 [00:00<?, ?it/s]

## Create a Vector Index

For more about the MognoDB Vector database, please read [What are Vector Databases?](https://www.mongodb.com/resources/basics/databases/vector-databases)
This code creates a vector index following the [MongoDB official document](https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#std-label-avs-types-vector-search).

In [9]:
# Create your index model, then create the search index

from pymongo.operations import SearchIndexModel
import time

search_index_model = SearchIndexModel(
  definition={
  "fields": [
    {
      "type": "vector",
      "path": "tweet.embedding",
      "numDimensions": 1536,
      "similarity": "cosine"
    }
  ]
},
  name="tweet_vector",
  type="vectorSearch"

)
result = tweet_collection.create_search_index(model=search_index_model)
print("New search index named " + result + " is building.")
# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None

if predicate is None:
  predicate = lambda index: index.get("queryable") is True

while True:
  indices = list(tweet_collection.list_search_indexes(result))
  if len(indices) and predicate(indices[0]):

    break
  time.sleep(5)

print(result + " is ready for querying.")

New search index named tweet_vector is building.
Polling to check if the index is ready. This may take up to a minute.
tweet_vector is ready for querying.


In [10]:
user_query = 'study ai'

for tweet in vector_search(user_query):
    print(tweet['tweet']['text'])

Kenya’s proposed Sovereign Wealth Fund Institute (SWFI) will introduce an election-year safeguard requiring that all funds under its control be audited and certified before every General Election, a move aimed at insulating public money from last-minute political spending.
RT @MariaSTsehai: There is no election to observe!
@goonerlicious @routinelytired @reddit_lies Any proof or are you retarded enough to think "duuuh number go up at time, must be stealing election"
RT @bloody_media: What the hell is happening in West Bengal? 😳😨 

A guy was caught in Nadia with over 10,000 voter cards! 

Was this a plan…
Idiot pretending to care. Roll on next election, then you won't be smiling 🤪 https://t.co/BWjK9W06uA
RT @RedEagleUpdates: Do you support a FULL investigation into CBS for interfering with the 2020 and 2024 Election ?

YES or NO ? https://t.…
RT @evenmaina: Buana I will live to testify that I have seen Tanzanians wenye walisema hatuna nidhamu juu ya kuprotest last year, protestin…
@this

## Retrieval-Augmented Generation (RAG) 

For more about RAG, please read [Retrieval-Augmented Generation (RAG) with Atlas Vector Search](https://www.mongodb.com/docs/atlas/atlas-vector-search/rag/#std-label-avs-rag).

In [11]:
from openai import OpenAI

delimiter = '###'
chat_model = 'gpt-4o'
temperature = 0

chat_history = [{"role": "system", "content": """ you are a chatbot, answer user questions based on the returned tweets."""}]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})
    
    tweets = vector_search(prompt)
    chat_history.append({"role": "system", "content": f" here the returned tweets delimited by {delimiter}{tweets}{delimiter}"})

    response = client.chat.completions.create(
        model=chat_model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [12]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  US election


Chatbot: Here are some thoughts and reactions from tweets about recent and upcoming US elections:

1. Discussions around the fairness of elections are ongoing, with some tweets suggesting concerns over past election integrity and future prospects. Notably, there are tweets expressing doubt about free and fair elections unless certain political figures are removed from influence.

2. There's a tweet indicating the 2020 election loss was attributed to the handling of COVID-19 and social unrest. It suggests that the Democratic party might struggle in the 2024 election due to perceived inaction.

3. One tweet mentions electoral depolarization in the 2024 election, specifically noting Trump's gains with immigrants and people of color, highlighting the complex demographic shifts in voter support.

4. There are grievances about the repetition of discussions on past elections, possibly feeling that certain issues are overstated or old narratives are being recycled.

5. A tweet sarcastically su

You:  election news


Chatbot: Here are some recent updates regarding elections from the tweets:

1. **Early Voting Numbers**: Early voting is underway for New York City's 2025 election, with almost 300,000 people having voted by the end of the fourth day.

2. **New Jersey Election**: There are multiple mentions of New Jersey election updates, with visits from Jeniffer González-Colón, the Governor of Puerto Rico, being highlighted.

3. **Election Monitoring**: There is news regarding federal monitors being sent to California by the Trump DOJ, which has elicited reactions from Governor Newsom.

4. **Local Campaigns**: The Jack Ciattarelli campaign in New Jersey seems to be gaining traction, with positive early voting numbers and less than a week until the election.

5. **Debate on Election Promises**: A tweet suggests that politicians who break election promises should not receive any salary or airtime, reflecting a sentiment of accountability.

These updates show active election activities, including early 

You:  quit


Chatbot: Goodbye!


## Reference

- *“Introducing Text and Code Embeddings.”* n.d. OpenAI. Accessed October 31, 2024. https://openai.com/index/introducing-text-and-code-embeddings/.
- *“What Are Vector Databases?”* n.d. MongoDB. Accessed October 31, 2024. https://www.mongodb.com/resources/basics/databases/vector-databases.
